In [4]:
import torch

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import torchvision
import torchvision.transforms.v2 as T
from torch.utils.data import DataLoader , TensorDataset
import torch.nn as nn
import torchmetrics

In [6]:
toTensor = T.Compose([T.ToImage() , T.ToDtype(torch.float32 , scale=True)])

train_and_valid_data = torchvision.datasets.FashionMNIST(
    root="datasets" , train=True , download=True , transform=toTensor
)

test_data = torchvision.datasets.FashionMNIST(
    root="datasets" , train=False , download=True , transform=toTensor
)

torch.manual_seed(42)
train_data , valid_data = torch.utils.data.random_split(
    train_and_valid_data , [55_000 , 5_000]
)

In [22]:
train_loader = DataLoader(train_data , shuffle=True , batch_size = 512 , pin_memory=True )
valid_loader = DataLoader(valid_data , shuffle=True , batch_size = 512 , pin_memory=True)
test_loader = DataLoader(test_data , shuffle=True , batch_size = 512 , pin_memory=True)




In [23]:
x_sample , y_sample = train_data[0]

In [24]:
y_sample

9

In [25]:
x_sample.shape

torch.Size([1, 28, 28])

In [26]:
x_sample.dtype

torch.float32

In [27]:
train_and_valid_data.classes[y_sample]

'Ankle boot'

In [28]:
for i in range(1 , 10):
    print(train_and_valid_data.classes[i] , end=" ")

Trouser Pullover Dress Coat Sandal Shirt Sneaker Bag Ankle boot 

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [30]:
n_epoch = 20

In [31]:
def eval(model , metric , val_loader):
    model.eval()
    metric.reset()
    
    with torch.no_grad():
        for x_batch , y_batch in val_loader:
            x_batch , y_batch = x_batch.to(device) , y_batch.to(device)
            
            y_pred = model(x_batch)
            
            metric.update(y_pred , y_batch)
            
    return metric.compute()





def train_eval(model , train_loader , criterion , optimizer ,
               val_loader , metric , n_epoch ):
    
    history = {"train_loss" : [] , "train_metric(accuracy)" : [] , "val_metric(accuracy)" : []}

    for epoch in range(n_epoch):
        model.train()
        metric.reset()
        total_loss = 0
        for x_batch , y_batch in train_loader:
            #data
            x_batch , y_batch = x_batch.to(device) , y_batch.to(device)
            
            #forward
            
            y_pred  = model(x_batch)
            
            #loss 
            
            loss = criterion(y_pred , y_batch)
            total_loss += loss.item()
            
            #optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            metric.update(y_pred , y_batch)
            
        loss_mean = total_loss/len(train_loader)
        
        history["train_loss"].append(loss_mean)
        history["train_metric(accuracy)"].append(metric.compute().item())
        
        history["val_metric(accuracy)"].append(eval(model , metric , val_loader).item())
        
        print(f"Epoch : {epoch + 1}/{n_epoch}, "
              f"Train Loss : {history['train_loss'][-1]}, "
              f"Train Metric ( Accuracy) : {history['train_metric(accuracy)'][-1]}, "
              f"Val Metric (accuracy) : {history['val_metric(accuracy)'][-1]}")
            

In [32]:
class ImageClassifier(nn.Module):
    def __init__(self , n_inputs , n_hidden1 , n_hidden2 , n_classes):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_inputs , n_hidden1) , 
            nn.ReLU(),
            nn.Linear(n_hidden1 , n_hidden2),
            nn.ReLU(),
            nn.Linear(n_hidden2 , n_classes)
        )
        
        
    def forward(self , x):
        return self.mlp(x)
    
torch.manual_seed(42)
model = ImageClassifier(n_inputs= 28 * 28 , n_hidden1=  300 , 
                        n_hidden2= 100 , n_classes= 10).to(device)

learning_rate = 0.1

sentropy = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters() , lr=learning_rate , momentum=0 )

accuracy_metric = torchmetrics.Accuracy(task='multiclass' , num_classes=10).to(device)

train_eval(model=model , 
           train_loader=train_loader,
           criterion=sentropy,
           optimizer=optimizer,
           val_loader=valid_loader,
           metric=accuracy_metric,
           n_epoch=n_epoch)
    

Epoch : 1/20, Train Loss : 1.406682523312392, Train Metric ( Accuracy) : 0.5595818161964417, Val Metric (accuracy) : 0.6424000263214111
Epoch : 2/20, Train Loss : 0.7617398550113043, Train Metric ( Accuracy) : 0.7130545377731323, Val Metric (accuracy) : 0.7508000135421753
Epoch : 3/20, Train Loss : 0.6311827169524299, Train Metric ( Accuracy) : 0.7703090906143188, Val Metric (accuracy) : 0.7767999768257141
Epoch : 4/20, Train Loss : 0.5653285764985614, Train Metric ( Accuracy) : 0.7989272475242615, Val Metric (accuracy) : 0.8082000017166138
Epoch : 5/20, Train Loss : 0.5227762217874881, Train Metric ( Accuracy) : 0.8143091201782227, Val Metric (accuracy) : 0.8184000253677368
Epoch : 6/20, Train Loss : 0.49167994023473177, Train Metric ( Accuracy) : 0.826090931892395, Val Metric (accuracy) : 0.8270000219345093
Epoch : 7/20, Train Loss : 0.4737724717016573, Train Metric ( Accuracy) : 0.8313272595405579, Val Metric (accuracy) : 0.8277999758720398
Epoch : 8/20, Train Loss : 0.4532129082966

In [38]:
model.eval()

x_new , y_new = next(iter(valid_loader))

print(len(valid_loader))
print(x_new.shape)
x_new = x_new[:3].to(device)
print(x_new.shape)

10
torch.Size([512, 1, 28, 28])
torch.Size([3, 1, 28, 28])


In [ ]:
with torch.no_grad():
    y_pred_logits = model(x_new)
    
    
    
print(y_pred_logits)

y_pred = y_pred_logits.argmax(dim=1)

print(y_pred)

tensor([[ 2.1723,  0.4753, 11.2694,  1.2055,  3.4685, -5.5942,  4.1923, -6.8598,
         -1.3340, -8.6228],
        [ 3.0385,  6.3782,  0.6215,  8.9629,  1.0980, -9.0665, -1.5528, -2.3363,
         -2.9354, -5.1512],
        [-2.0960, -3.5928, -5.3018, -4.1145, -5.2092,  5.2536, -3.4882,  7.9512,
          2.8521, 11.4585]], device='cuda:0')
tensor(29, device='cuda:0')
